# Plotting XTRRA RHI data in Py-ART

Objectives:
1. Load and plot data from the XTRRA for a recent rain event (RHI scans)

Remember Py-ART? Let's use it now to plot some data from our own local radar, the X-band Teaching and Research Radar (XTRRA).

Make sure the files, pyXTRRA.py, and xtrra_config.py live in the same directory as this notebook.

## Import necessary libraries and the magic "inline" command. 

In [3]:
import pyart
import os
import sys
import glob
import copy
import datetime
import numpy as np
from matplotlib import pyplot as plt
#import cartopy.crs as ccrs
#import cartopy.feature as cfeat
#%matplotlib inline

# Some colleagues and I wrote this module specifically for importing XTRRA data, which isn't CF-Radial compliant.
from pyPIPS.pyXTRRA import *

# These lines tell Python where a shapefile containing county boundaries is located.
#shapefile_directory = '/class/rtanama/data/EAPS523-2020/shapefiles/'
# Uncomment the following line if the line above doesn't work.
# shapefile_directory = '/depot/eapsdept/data/52300/shapefiles/'
#countyshapefile = os.path.join(shapefile_directory, 'countyl010g.shp')
#counties = cfeat.ShapelyFeature(Reader(countyshapefile).geometries(), ccrs.PlateCarree(),
#                                facecolor='none')
                                
# Finally, create some directories inside this folder for storing the graphical output.
# os.system('mkdir -pv images/')
#os.system('mkdir -pv images/ppi/')
#os.system('mkdir -pv images/rhi/')

0

# Now, let's load up the RHI scans.
On this day, I briefly interrupted the VCP 215 scanning pattern to perform some true RHI (vertical) scans. Remember that in an RHI (range-height indicator) scan, the azimuth stays constant, while the elevation varies (from 0 to 60 degrees, in this case). You will notice that the times for the RHI scans are about 5 minutes after the PPI volume that we plotted. The syntax for reading in the files is similar to that for the PPI scans.

In [6]:
rhiDir = '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/'
image_subdir = 'images/rhi'
image_fulldir = os.path.join(rhiDir, image_subdir)
if not os.path.exists(image_fulldir):
    os.makedirs(image_fulldir)
rhiFiles = glob.glob(rhiDir + '*.netcdf') # Puts the names of files at the specified path into a list
# Uncomment the following line if the line above doesn't work.
#rhiFiles = glob.glob('/depot/eapsdept/data/52300/rhivol/*.netcdf')
rhiFiles.sort()  # Puts the files in order by name/time
rhiFiles   # List the file names to the display. There should be 16 of them.

['/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170016.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170037.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170057.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170117.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170138.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170158.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170218.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTRRA_20190319_rhi_over_house/purdue.in-20200319-170238.netcdf',
 '/Users/dawson29/sshfs_mounts/depot/data/from_Robin/XTR

Let's create a list of Radar class objects, one per slice.

In [7]:
RHI = []  # Initialize an empty list of Radar objects.
for file in rhiFiles:
    RHI.append(CreateRadarFromRXM25netCDF(file, instrument_name = 'XTRRA'))
RHI # List the radar objects, one per sweep.

List the contents of each RHI file.

In [8]:
RHI[0].fields.keys()

dict_keys(['reflectivity', 'normalized_coherent_power', 'spectral_width', 'velocity', 'corrected_reflectivity', 'differential_reflectivity', 'differential_phase', 'specific_differential_phase', 'corrected_differential_reflectivity', 'signal_to_noise_ratio', 'rain_rate', 'cross_correlation_ratio'])

The code to plot the RHI scans looks a little bit different, however.

In [9]:
RHIDisplay = [] # Initialize an empty list
for rhi in RHI:
    RHIDisplay.append(pyart.graph.RadarMapDisplay(rhi))

In [26]:
variable = 'reflectivity' # The variable to plot
for index in range(len(RHIDisplay)): # Iterate over the number of sweeps
    plt.figure()  # Make a new figure for each sweep.
    RHIDisplay[index].plot_rhi(variable, cmap = 'pyart_HomeyerRainbow',title = 'RHI')
    plt.xlim([0, 20])
    plt.ylim([0, 8]) # Restricts the vertical axis to 0-8 km AGL.
    plt.axes().set_aspect('equal')
    # The following line saves a PNG of the image, using the same file name prefix.
    plt.savefig(image_fulldir + '/' + rhiFiles[index][-32:-7] + '.' + variable + '.png') # Save the plots
    plt.close() # You may wish to uncomment this line if the notebook becomes too large.

/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys


In [23]:
variable = 'velocity' # The variable to plot
for index in range(len(RHIDisplay)): # Iterate over the number of sweeps
    plt.figure()  # Make a new figure for each sweep.
    RHIDisplay[index].plot_rhi(variable, cmap = 'seismic',title = 'RHI')
    plt.xlim([0, 20])
    plt.ylim([0, 8]) # Restricts the vertical axis to 0-8 km AGL.
    plt.axes().set_aspect('equal')
    # The following line saves a PNG of the image, using the same file name prefix.
    plt.savefig(image_fulldir + '/' + rhiFiles[index][-32:-7] + '.' + variable + '.png') # Save the plots
    plt.close() # You may wish to uncomment this line if the notebook becomes too large.

/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys


In [27]:
variable = 'corrected_differential_reflectivity' # The variable to plot
for index in range(len(RHIDisplay)): # Iterate over the number of sweeps
    plt.figure()  # Make a new figure for each sweep.
    RHIDisplay[index].plot_rhi(variable, cmap = 'pyart_HomeyerRainbow',title = 'RHI')
    plt.xlim([0, 20])
    plt.ylim([0, 8]) # Restricts the vertical axis to 0-8 km AGL.
    plt.axes().set_aspect('equal')
    # The following line saves a PNG of the image, using the same file name prefix.
    plt.savefig(image_fulldir + '/' 
                + rhiFiles[index][-32:-7] + '.' + variable + '.png') # Save the plots
    plt.close() # You may wish to uncomment this line if the notebook becomes too large.

/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys


In [25]:
variable = 'cross_correlation_ratio' # The variable to plot
for index in range(len(RHIDisplay)): # Iterate over the number of sweeps
    plt.figure()  # Make a new figure for each sweep.
    RHIDisplay[index].plot_rhi(variable, cmap = 'pyart_HomeyerRainbow',title = 'RHI', vmin = 0.5, vmax = 1.0)
    plt.xlim([0, 20])
    plt.ylim([0, 8]) # Restricts the vertical axis to 0-8 km AGL.
    plt.axes().set_aspect('equal')
    # The following line saves a PNG of the image, using the same file name prefix.
    plt.savefig(image_fulldir + '/' + rhiFiles[index][-32:-7] + '.' + variable + '.png') # Save the plots
    plt.close() # You may wish to uncomment this line if the notebook becomes too large.

/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dawson29/miniconda3/envs/pyPIPS/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys


In [ ]:
# Print the azimuth of the RHI
for index in range(len(RHI)):
    print(rhiFiles[index][-32:-7] + " azimuth: " + str(np.mean(RHI[index].azimuth['data'])) + " deg")